In [1]:
# export
from dotenv import load_dotenv
load_dotenv()

import os
import pickle
from datetime import datetime


import pandas as pd
import numpy as np
from datetime import datetime
from binance.client import Client
from binance.websockets import BinanceSocketManager
from twisted.internet import reactor
from datetime import datetime
import btalib


#cor coinmarketcapapi
from coinmarketcapapi import CoinMarketCapAPI, CoinMarketCapAPIError
from requests import Request, Session
from requests.exceptions import ConnectionError, Timeout, TooManyRedirects
import json

#custom library
import MLTrain as mlt

pd.set_option('display.max_rows', 500)


import logging

# Gets or creates a logger
logger = logging.getLogger(__name__)  

# set log level
logger.setLevel(logging.INFO)

# define file handler and set formatter
file_handler = logging.FileHandler('logfile.log')
formatter    = logging.Formatter('%(asctime)s : %(levelname)s : %(name)s : %(message)s')
file_handler.setFormatter(formatter)

# add file handler to logger
logger.addHandler(file_handler)

In [8]:
# export
def get_coin_market_cap_data():
    url = 'https://pro-api.coinmarketcap.com/v1/cryptocurrency/listings/latest'
    parameters = {
      'start':'1',
      'limit':'1000',
      'convert':'USD'
    }
    headers = {
      'Accepts': 'application/json',
      'X-CMC_PRO_API_KEY': os.environ.get('COIN_MARKET_CAP_API_KEY'),
    }

    session = Session()
    session.headers.update(headers)

    try:
      logger.info('try to connect {url}'.format(url=url))  
      response = session.get(url, params=parameters)
      data = json.loads(response.text)
      #print(data)
    except (ConnectionError, Timeout, TooManyRedirects) as e:
      print(e)
      logger.error('Can NOT connect {url}'.format(url=url))


    now = datetime.now()
    dt_string = now.strftime("%Y%m%d%H%M")

    listx = []
    for coin_dict in data.get("data"):
        listx.append([ \
            coin_dict.get('name'),
            coin_dict.get('symbol'),
            coin_dict.get('cmc_rank'),
            coin_dict.get('quote').get('USD').get('volume_24h'),
            coin_dict.get('quote').get('USD').get('market_cap'),
            coin_dict.get('quote').get('USD').get('last_updated'),
            dt_string          
        ])
    
    return listx


def append_coin_market_cap_data(v_list):
    file = open("cmc_data.pkl","rb")
    list_old = pickle.load(file)
    
    list_all = list_old + v_list
    
    pickle_out = open("cmc_data.pkl","wb")
    pickle.dump(list_all, pickle_out)
    pickle_out.close()


In [3]:
# export
def f_get_coinlist():
    coin_list_df = pd.read_csv("crypto_lu.csv")
    coin_list_df["Symbolusd"] =  coin_list_df["Symbol"].apply(lambda x: x + "USDT" )
    coin_list_df["Keywords"]  = coin_list_df.apply(lambda row: row["Name"].lower() + "," + 
                                   row["Symbol"].lower() + "," + 
                                   row["Symbolusd"].lower() + "," + 
                                   row["Keywords"].lower()  , axis = 1)


    coin_list_df = coin_list_df.head(25)

    coin_list = list(coin_list_df.Symbolusd)
    return coin_list


def f_get_binance_client():
    api_key = os.environ.get('BINANCE_API')
    api_secret = os.environ.get('BINANCE_SECRET')
    return Client(api_key, api_secret)


def f_load_allcoins(coin_list = None):
    
    client = f_get_binance_client()
    coin_list = f_get_coinlist()
    dict_all = {}
    for coin in coin_list:  
        dict_coin = {}    
        logger.info('Initial load {x}'.format(x=coin))
        try:
            klines = client.get_historical_klines(coin, Client.KLINE_INTERVAL_2HOUR, "365 day ago UTC")
        except Exception as e:
            print(str(e))
            logger.error('Initial load {x} FAILED'.format(x=coin))
            continue
        for x in klines:
            ts = int(x[0]) / 1000
            dt = datetime.utcfromtimestamp(ts).strftime('%Y-%m-%d %H:%M:%S')
            dict_coin.update({dt:[x[0], x[4], x[5], x[8]]}) #date, price, volume, numoftrades    
        dict_all.update({coin:dict_coin})

    return dict_all



# fetch 1 minute klines for the last day up until now
def f_update_lastNdays(dict_all=None):
    
    if dict_all is None:
        file = open("dict_all.pkl","rb")
        dict_all = pickle.load(file)
    
    client = f_get_binance_client()
    for coin in list(dict_all.keys()):  
        dict_coin = {}
        logger.info('try to update coin {x}'.format(x=coin))
        
        try:
            klines = client.get_historical_klines(coin, Client.KLINE_INTERVAL_2HOUR, "14 day ago UTC")
        except Exception as e:
            print(str(e))
            logger.error('UPDATE list of coin: {x} FAILED, seems no info'.format(x=coin))
            ##Make warning to say that such coin data is not up-to-date!!
            continue        
        dict_coin = dict_all.get(coin)
        for x in klines:
            ts = int(x[0]) / 1000
            dt = datetime.utcfromtimestamp(ts).strftime('%Y-%m-%d %H:%M:%S')
            dict_coin.update({dt:[x[0], x[4], x[5], x[8]]}) #date, price, volume, numoftrades    
        dict_all.update({coin:dict_coin})
        
    pickle_out = open("dict_all.pkl","wb")
    pickle.dump(dict_all, pickle_out)
    pickle_out.close()

    return dict_all



# fetch 1 minute klines for the last day up until now
def f_update_lastprice(dict_all=None):
    
    client = f_get_binance_client()
    for coin in list(dict_all.keys()):  
        dict_coin = {}
        logger.info('try to update coin {x}'.format(x=coin))
        
        try:
            klines = client.get_historical_klines(coin, Client.KLINE_INTERVAL_1MINUTE, "1 minute ago UTC")
        except Exception as e:
            print(str(e))
            logger.error('UPDATE list of coin: {x} FAILED, seems no info'.format(x=coin))
            ##Make warning to say that such coin data is not up-to-date!!
            continue        
        dict_coin = dict_all.get(coin)
        for x in klines:
            ts = int(x[0]) / 1000
            dt = datetime.utcfromtimestamp(ts).strftime('%Y-%m-%d %H:%M:%S')
            dict_coin.update({dt:[x[0], x[4], x[5], x[8]]}) #date, price, volume, numoftrades    
        dict_all.update({coin:dict_coin})

    return dict_all



def f_get_all_price(dict_all_coins):
    #dict_all_coins = f_load_allcoins(coin_list = None)
    #dict_all_coins = f_update_lastprice(dict_all=dict_all_coins)
    
    df_all_coins = pd.DataFrame(columns= ["coin", "dt", "price", "vol", "tradefreq"])
    for coin in dict_all_coins.keys():
        dict_coin = dict_all_coins.get(coin)
        listx = []
        for dt in dict_coin.keys():
            listx.append([coin, dt, dict_coin.get(dt)[1], dict_coin.get(dt)[2], dict_coin.get(dt)[3]])
        df_all_coins = df_all_coins.append(pd.DataFrame(data = listx,
                                         columns= ["coin", "dt", "price", "vol", "tradefreq"]
                                        )
                           )
    df_all_coins.dt = df_all_coins.dt.apply(lambda x: datetime.strptime(str(x), '%Y-%m-%d %H:%M:%S')  )
    df_all_coins['last_time'] = 0
    for coin in df_all_coins.coin.unique():
        max_dt = np.max(df_all_coins.loc[df_all_coins.coin == coin, 'dt'])
        df_all_coins.loc[(df_all_coins.coin == coin) & (df_all_coins.dt == max_dt ) , 'last_time'] = 1

    df_all_coins.set_index("dt", drop=True, inplace = True )
    df_all_coins.sort_index(inplace=True)
    df_all_coins["is_day_end"] = 0
    df_all_coins.loc[[True if x.strftime("%H:%M:%S") == '00:00:00' else False for x in list(df_all_coins.index)], "is_day_end"] = 1  
    df_all_coins.price = df_all_coins.price.astype(float)
    df_all_coins.tradefreq= df_all_coins.tradefreq.astype(float)
    df_all_coins.vol= df_all_coins.vol.astype(float)
    
    
    return df_all_coins
    


def f_calc_indicators(df):
    df_all_coins = df.copy()
    
    #df_all_coins = f_get_all_price()
    
    for coin in df_all_coins.coin.unique():
        index_t = (df_all_coins.coin == coin)
        lenx = len(df_all_coins.loc[df_all_coins.coin == coin, :])
        if  lenx < 100:
            print("{coin} lacks of enough sample {sample} so lets drop it!!".format(coin=coin, sample=lenx))
            continue
        #df_all_coins.loc[index_t, 'ma12'] = btalib.sma(df_all_coins.loc[index_t, 'price'], period=12).df.values.tolist()
        #df_all_coins.loc[index_t, 'ma24'] = btalib.sma(df_all_coins.loc[index_t, 'price'], period=24).df.values.tolist()
        #df_all_coins.loc[index_t, 'ma48'] = btalib.sma(df_all_coins.loc[index_t, 'price'], period=48).df.values.tolist()
        #df_all_coins.loc[index_t, 'ma148'] = btalib.sma(df_all_coins.loc[index_t, 'price'], period=96).df.values.tolist()
        df_all_coins.loc[index_t, 'rsi14'] = btalib.rsi(df_all_coins.loc[index_t, 'price'], period=14).df.values.tolist()
        df_all_coins.loc[index_t, 'rsi28'] = btalib.rsi(df_all_coins.loc[index_t, 'price'], period=28).df.values.tolist()
        df_all_coins.loc[index_t, 'rsi42'] = btalib.rsi(df_all_coins.loc[index_t, 'price'], period=42).df.values.tolist()
        df_all_coins.loc[index_t, 'rsi48'] = btalib.rsi(df_all_coins.loc[index_t, 'price'], period=48).df.values.tolist()
        df_all_coins.loc[index_t, 'rsi72'] = btalib.rsi(df_all_coins.loc[index_t, 'price'], period=72).df.values.tolist()
        df_all_coins.loc[index_t, ["macd12", "signal12", "hist12"] ] =  \
                        btalib.macd(df_all_coins.loc[index_t, 'price'], pfast=12, pslow=24, psignal=9).df.values.tolist()
        df_all_coins.loc[index_t, ["macd6", "signal6", "hist6"] ] =  \
                        btalib.macd(df_all_coins.loc[index_t, 'price'], pfast=6, pslow=12, psignal=4).df.values.tolist()

    df_all_coins['macd12_ratio'] = df_all_coins['macd12'] / df_all_coins['price']
    df_all_coins['signal12_ratio'] = df_all_coins['signal12'] / df_all_coins['price']
    
    
    return df_all_coins


def f_get_last_minute_rows(v_df, rnk = 1):
    #df = f_calc_indicators()
    df = v_df.copy()
    df.reset_index(inplace=True)
    df.loc[:, "rnk_dt"] =  df.groupby("coin")["dt"].rank("dense", ascending=False)
    df = df.loc[df.rnk_dt == rnk, :]
    df.set_index("dt", inplace = True)
    df.drop(['rnk_dt'], axis=1, inplace = True)
    return df
    

In [4]:
# export
def set_target(row, margin):
    meanx = np.nanmean([row["next1price"], row["next2price"], row["next3price"] ])
    if meanx > row["price"] * (1.00 + margin):
        return 1
    elif meanx < row["price"] * (1.00 - margin):
        return -1
    else:
        return 0

    
def set_target_numeric(row):
    return np.nanmean([row["next1price"], row["next2price"], row["next3price"] ])    
    
def f_transform_df_target(df_all_coins, shift_period = 2):    
    Target_Type = 'Binary'
    df_all_coins["next1price"] = df_all_coins.groupby(['coin'])['price'].shift(-1*shift_period)
    df_all_coins["next2price"] = df_all_coins.groupby(['coin'])['price'].shift(-2*shift_period)
    df_all_coins["next3price"] = df_all_coins.groupby(['coin'])['price'].shift(-4*shift_period)

    df_all_coins["Target"] = 0
       

    
    if Target_Type == 'Numeric':    
        #Numeric Target
        df_all_coins.loc[df_all_coins.last_time == 0, "Target"] = \
                df_all_coins.loc[df_all_coins.last_time == 0, :].apply(lambda row: set_target_numeric(row), axis = 1)    
    else:
        #Binary Target
        df_all_coins.loc[df_all_coins.last_time == 0, "Target"] = \
                df_all_coins.loc[df_all_coins.last_time == 0, :].apply(lambda row: set_target(row, 0.02), axis = 1)
    
    pickle_out = open("df_all.pkl","wb")
    pickle.dump(df_all_coins, pickle_out)
    pickle_out.close()
    
    return df_all_coins


def f_prep_df_to_ML(df):
    df_tmp = df.copy()
    df_tmp.dropna(inplace = True)
    y = df_tmp.loc[:, ["Target"]]
    X = df_tmp.loc[:, [ x for x in df_tmp.columns if x not in (["Target" 
                                                                ,"coin" 
                                                                ,"next1price" 
                                                                ,"next2price" 
                                                                ,"next3price" 
                                                                ,"price"
                                                               # ,"is_day_end"
                                                               # ,"last_time"
                                                               ])  ]]

    return X, y 

In [34]:
dict_init = f_load_allcoins()

APIError(code=-1121): Invalid symbol.
APIError(code=-1121): Invalid symbol.
APIError(code=-1121): Invalid symbol.


In [5]:
dict_tmp = f_update_lastNdays()
dict_tmp = f_update_lastprice(dict_tmp)
df = f_get_all_price(dict_tmp)

In [7]:
df = f_calc_indicators(df)
df = f_transform_df_target(df, shift_period = 2)

/root/projects/binance_first/venv/lib/python3.6/site-packages/ipykernel_launcher.py:3: RuntimeWarning: Mean of empty slice
  This is separate from the ipykernel package so we can avoid doing imports until


In [6]:
df

,coin,price,vol,tradefreq,last_time,is_day_end
dt,,,,,,
2020-05-10 18:00:00,BTCUSDT,8502.94000,7.530685e+03,77046.0,0,0
2020-05-10 18:00:00,BNBUSDT,14.96010,1.456877e+05,8205.0,0,0
2020-05-10 18:00:00,BUSDUSDT,0.99980,2.427941e+06,8947.0,0,0
2020-05-10 18:00:00,XMRUSDT,57.65000,1.061929e+04,2733.0,0,0
2020-05-10 18:00:00,XLMUSDT,0.06192,2.733797e+07,6510.0,0,0
...,...,...,...,...,...,...
2021-05-22 19:01:00,SOLUSDT,32.70100,5.093227e+03,306.0,1,0
2021-05-22 19:01:00,ADAUSDT,1.45380,3.492720e+05,790.0,1,0
2021-05-22 19:01:00,USDCUSDT,0.99930,6.419792e+04,60.0,1,0


In [10]:
#def f_calc_rank_change(df):
file = open("cmc_data.pkl","rb")
list_old = pickle.load(file)
list_old

[['Bitcoin',
  'BTC',
  1,
  103652676043.30545,
  908537554719.66,
  '2021-05-13T20:04:02.000Z',
  '202105132020'],
 ['Ethereum',
  'ETH',
  2,
  84516571944.66193,
  421382960950.4562,
  '2021-05-13T20:04:02.000Z',
  '202105132020'],
 ['Binance Coin',
  'BNB',
  3,
  7539633216.036843,
  84346343583.18658,
  '2021-05-13T20:03:14.000Z',
  '202105132020'],
 ['Cardano',
  'ADA',
  4,
  13800709366.825464,
  59534461706.086716,
  '2021-05-13T20:03:16.000Z',
  '202105132020'],
 ['Tether',
  'USDT',
  5,
  246239035545.35406,
  57745988686.10043,
  '2021-05-13T20:03:14.000Z',
  '202105132020'],
 ['Dogecoin',
  'DOGE',
  6,
  13851898830.258604,
  49652409103.65721,
  '2021-05-13T20:04:03.000Z',
  '202105132020'],
 ['XRP',
  'XRP',
  7,
  12793888388.41379,
  44063337854.109795,
  '2021-05-13T20:04:02.000Z',
  '202105132020'],
 ['Polkadot',
  'DOT',
  8,
  7244276461.974504,
  36855688126.50413,
  '2021-05-13T20:04:07.000Z',
  '202105132020'],
 ['Internet Computer',
  'ICP',
  9,
  88894945

In [17]:
#def f_calc_rank_change(df):
file = open("cmc_data.pkl","rb")
list_old = pickle.load(file)

df_all_coins = pd.DataFrame(data = list_old, columns= ['coin', 
                                        'symbol',
                                        'cmc_rank',
                                        'volume_24h',
                                        'market_cap',
                                        'last_updated',
                                        'dt_string'
                                        ])

df_all_coins['dt'] = df_all_coins['dt_string'].apply(lambda x: datetime.strptime(x, '%Y%m%d%H%M%S') ) 
df_all_coins = df_all_coins.sort_values(by = 'dt')

for coin in df_all_coins.coin.unique():
    index_t = (df_all_coins.coin == coin)
    lenx = len(df_all_coins.loc[df_all_coins.coin == coin, :])
    if  lenx < 20:
        print("{coin} lacks of enough sample {sample} so lets drop it!!".format(coin=coin, sample=lenx))
        continue
    df_all_coins.loc[index_t, 'ma4'] = btalib.sma(df_all_coins.loc[index_t, 'cmc_rank'], period=4).df.values.tolist()
    df_all_coins.loc[index_t, 'ma18'] = btalib.sma(df_all_coins.loc[index_t, 'cmc_rank'], period=18).df.values.tolist()
    

df_all_coins.loc[:, "rnk_dt"] =  df.groupby("coin")["dt"].rank("dense", ascending=False)
df_last = df_all_coins.loc[df_all_coins.rnk_dt == 1, :]
    #return df_all_coins





Bounce Token (OLD) lacks of enough sample 11 so lets drop it!!
Alpaca Finance lacks of enough sample 15 so lets drop it!!
EUNO lacks of enough sample 16 so lets drop it!!
AnRKey X lacks of enough sample 2 so lets drop it!!
PANTHEON X lacks of enough sample 3 so lets drop it!!
Urus lacks of enough sample 3 so lets drop it!!
BitGreen lacks of enough sample 6 so lets drop it!!
Raze Network lacks of enough sample 18 so lets drop it!!
Garlicoin lacks of enough sample 7 so lets drop it!!
Shard lacks of enough sample 18 so lets drop it!!
Kattana lacks of enough sample 18 so lets drop it!!
SeChain lacks of enough sample 17 so lets drop it!!
Noku lacks of enough sample 8 so lets drop it!!
ETNA Network lacks of enough sample 1 so lets drop it!!
COVIR.IO lacks of enough sample 6 so lets drop it!!
Tower token lacks of enough sample 3 so lets drop it!!
CryptalDash lacks of enough sample 1 so lets drop it!!
Tokoin lacks of enough sample 4 so lets drop it!!
carVertical lacks of enough sample 6 so let

KeyError: 'coin'

In [94]:
df_all_coins.loc[:, "rnk_dt"] =  df_all_coins.groupby("coin")["dt"].rank("dense", ascending=False)
df_last = df_all_coins.loc[df_all_coins.rnk_dt == 1, :]
df_last["rank_delta"] = df_last.apply(lambda row: row["ma18"] - row["ma4"], axis = 1 ) 
df_last["log_cmc"] = df_last["cmc_rank"].apply(lambda x: math.log(x, 20))
df_last[(df_last["rank_delta"] * np.exp(-1 * df_last["log_cmc"]) >= 3)
        & 
        (df_last["cmc_rank"] <= df_last["ma4"] )]



/root/projects/binance_first/venv/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/root/projects/binance_first/venv/lib/python3.6/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


,coin,symbol,cmc_rank,volume_24h,market_cap,last_updated,dt_string,dt,ma4,ma18,rnk_dt,rank_delta,log_cmc
70668,Waves Enterprise,WEST,669,9.085932e+05,2.287658e+07,2021-05-22T18:02:03.000Z,202105221802,2021-05-22 18:00:02,689.75,720.166667,1.0,30.416667,2.171684
70679,StableXSwap,STAX,680,1.729179e+04,2.235587e+07,2021-05-22T18:02:05.000Z,202105221802,2021-05-22 18:00:02,680.75,708.555556,1.0,27.805556,2.177128
70627,AC Milan Fan Token,ACM,628,3.096302e+07,2.667881e+07,2021-05-22T18:02:05.000Z,202105221802,2021-05-22 18:00:02,640.75,666.722222,1.0,25.972222,2.150573
70630,Unicly CryptoPunks Collection,UPUNK,631,2.249569e+05,2.627453e+07,2021-05-22T18:02:04.000Z,202105221802,2021-05-22 18:00:02,636.00,665.611111,1.0,29.611111,2.152164
70645,Gulden,NLG,646,1.275037e+06,2.471407e+07,2021-05-22T18:02:05.000Z,202105221802,2021-05-22 18:00:02,650.00,703.555556,1.0,53.555556,2.160006
70647,Newton,NEW,648,5.901318e+07,2.506002e+07,2021-05-22T18:01:09.000Z,202105221802,2021-05-22 18:00:02,701.75,816.333333,1.0,114.583333,2.161038
70734,Cobak Token,CBK,735,4.224857e+04,1.742025e+07,2021-05-22T18:01:07.000Z,202105221802,2021-05-22 18:00:02,739.50,813.722222,1.0,74.222222,2.203091
70744,Trias (old),TRY,745,3.763263e+00,1.672297e+07,2021-05-22T18:02:07.000Z,202105221802,2021-05-22 18:00:02,745.75,779.944444,1.0,34.194444,2.207602
70697,ICHI,ICHI,698,6.380103e+04,2.044743e+07,2021-05-22T18:02:07.000Z,202105221802,2021-05-22 18:00:02,716.00,749.722222,1.0,33.722222,2.185849
70533,Project Pai,PAI,534,9.679674e+06,3.782035e+07,2021-05-22T18:01:10.000Z,202105221802,2021-05-22 18:00:02,564.50,626.888889,1.0,62.388889,2.096448


In [91]:
df_last

,coin,symbol,cmc_rank,volume_24h,market_cap,last_updated,dt_string,dt,ma4,ma18,rnk_dt,rank_delta
2984,PANTHEON X,XPN,985,0.000000e+00,1.240986e+07,2021-05-13T20:53:08.000Z,202105132053,2021-05-13 20:05:03,NaN,NaN,1.0,NaN
2994,Urus,URUS,995,1.740772e+06,1.205948e+07,2021-05-13T20:53:06.000Z,202105132053,2021-05-13 20:05:03,NaN,NaN,1.0,NaN
2996,AnRKey X,$ANRX,997,2.267023e+05,1.201067e+07,2021-05-13T20:52:10.000Z,202105132053,2021-05-13 20:05:03,NaN,NaN,1.0,NaN
3862,ETNA Network,ETNA,863,2.012617e+06,1.802534e+07,2021-05-13T23:56:09.000Z,202105132357,2021-05-13 23:05:07,NaN,NaN,1.0,NaN
5993,BitGreen,BITG,994,1.129179e+06,1.233835e+07,2021-05-14T05:57:06.000Z,202105140557,2021-05-14 05:05:07,NaN,NaN,1.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
70338,MX Token,MX,339,4.066046e+07,8.804942e+07,2021-05-22T18:01:08.000Z,202105221802,2021-05-22 18:00:02,334.50,307.777778,1.0,-26.722222
70339,Firo,FIRO,340,6.974666e+06,8.684941e+07,2021-05-22T18:02:05.000Z,202105221802,2021-05-22 18:00:02,336.00,326.166667,1.0,-9.833333
70340,SUN,SUN,341,2.741422e+07,8.615268e+07,2021-05-22T18:01:09.000Z,202105221802,2021-05-22 18:00:02,351.50,344.888889,1.0,-6.611111
70327,IRISnet,IRIS,328,2.276575e+07,9.114880e+07,2021-05-22T18:01:10.000Z,202105221802,2021-05-22 18:00:02,353.50,352.055556,1.0,-1.444444


In [86]:
import math
df_all_coins.loc[df_all_coins.coin == 'IRISnet',:]
np.exp(-1 * np.log10(2*10))
#np.exp(-1.5 * math.log(10,20) )
#dt_obj = datetime.fromtimestamp(timestamp)

0.2722512309825426

In [22]:
dt_obj = datetime.strptime('202105132020', '%Y%m%d%H%M%S') 

In [13]:
df = f_calc_indicators(df)
df = f_transform_df_target(df)

SHIBUSDT lacks of enough sample 26 so lets drop it!!


/root/projects/binance_first/venv/lib/python3.6/site-packages/ipykernel_launcher.py:3: RuntimeWarning: Mean of empty slice
  This is separate from the ipykernel package so we can avoid doing imports until


In [14]:
df.loc[df.coin == 'SHIBUSDT', :]

,coin,price,vol,tradefreq,last_time,is_day_end,rsi14,rsi28,rsi42,rsi48,...,hist12,macd6,signal6,hist6,macd12_ratio,signal12_ratio,next1price,next2price,next3price,Target
dt,,,,,,,,,,,,,,,,,,,,,
2021-05-10 10:00:00,SHIBUSDT,0.000030,1.527431e+13,730457.0,0,0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,0.000029,0.000030,0.000035,1
2021-05-10 12:00:00,SHIBUSDT,0.000031,2.137269e+13,1037643.0,0,0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,0.000027,0.000031,0.000034,0
2021-05-10 14:00:00,SHIBUSDT,0.000029,9.380247e+12,508738.0,0,0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,0.000030,0.000035,0.000031,1
2021-05-10 16:00:00,SHIBUSDT,0.000027,1.156413e+13,528332.0,0,0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,0.000031,0.000034,0.000032,1
2021-05-10 18:00:00,SHIBUSDT,0.000030,8.962142e+12,437754.0,0,0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,0.000035,0.000035,0.000033,1
2021-05-10 20:00:00,SHIBUSDT,0.000031,9.314410e+12,469977.0,0,0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,0.000034,0.000034,0.000031,1
2021-05-10 22:00:00,SHIBUSDT,0.000035,1.943438e+13,864728.0,0,0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,0.000035,0.000031,0.000030,-1
2021-05-11 00:00:00,SHIBUSDT,0.000034,9.034212e+12,414005.0,0,1,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,0.000034,0.000032,0.000032,-1
2021-05-11 02:00:00,SHIBUSDT,0.000035,6.691246e+12,357273.0,0,0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,0.000031,0.000033,0.000033,-1


In [28]:
df1 = f_get_last_minute_rows(df, rnk = 15)

df1.loc[df1.coin == 'SHIBUSDT', :]

X = mlt.f_prep_df_to_ML(df1.loc[df1.coin == 'SHIBUSDT', :], is_train = False)[0]

X

,vol,tradefreq,last_time,is_day_end,rsi14,rsi28,rsi42,rsi48,rsi72,macd12,signal12,hist12,macd6,signal6,hist6,macd12_ratio,signal12_ratio
dt,,,,,,,,,,,,,,,,,


In [56]:
X

,vol,tradefreq,last_time,is_day_end,rsi14,rsi28,rsi42,rsi48,rsi72,macd12,signal12,hist12,macd6,signal6,hist6,macd12_ratio,signal12_ratio
dt,,,,,,,,,,,,,,,,,
2021-05-07 20:00:00,1.507296e+09,867978.0,0,0,67.285092,65.872928,65.037728,64.713876,63.743638,0.028953,0.030047,-0.001094,0.015661,0.010977,0.004684,0.042723,0.044337


In [21]:
file = open("model_dict.pkl","rb")
model_dict = pickle.load(file)
model_dict.get("DOGEUSDT")[0].predict_proba(X)

array([[0.29072402, 0.10066178, 0.6086142 ]])

In [66]:
from exporter import export
export("Financial_Indicators.ipynb","Financial_Indicators.py")

In [ ]:
# export
if __name__ == "__main__":
    print("main")